In [13]:
%load_ext autoreload
%autoreload 2
%matplotlib notebook

import pandas as pd
import numpy as np
from helpers import (
    load_data,
    EMG_CHANNELS,
    EMG_HP_CHANNELS,
    GYRO_CHANNELS,
    ACC_CHANNELS,
    IMU_CHANNELS,
)
import itertools
from feature_extraction import (
    FeatureExtractor,
    process_dataframe,
    zero_crossings,
    mean_crossings,
    spectral_entropy
)
from sklearn.decomposition import KernelPCA
from sklearn.cluster import DBSCAN
from mpl_toolkits.mplot3d import Axes3D

from matplotlib import pyplot as plt

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [14]:
dataset = load_data()
dataset

Trimming observations to 967 samples


time_ms       ch0       ch1  ch0_hp  ch1_hp  \
body_label rep sample_num                                                    
0          1   0.0         1514824.503  12535249  12566283  -11889   17295   
               1.0         1514827.496  12536264  12559246   -2063   13384   
               2.0         1514830.493  12538584  12565279    2757   16008   
               3.0         1514833.500  12546745  12567024    7504    5644   
               4.0         1514836.498  12537375  12545467   -3855  -15893   
...                                ...       ...       ...     ...     ...   
4          3   962.0       1618110.457  12136089  12370189  -31898   45780   
               963.0       1618113.448  12165574  12340495   20709    2953   
               964.0       1618116.437  12200077  12301532   45799  -36427   
               965.0       1618119.442  12153545  12317293  -14411   -3002   
               966.0       1618122.435  12099585  12324114  -50651   13979   

                                 qx        qy        qz        qw          gx  \
body_label rep sample_num                                                       
0          1   0.0         0.321960 -0.596619 -0.621826  0.392090   -1.342870   
               1.0         0.321960 -0.596741 -0.621826  0.392029   -1.510729   
               2.0         0.321960 -0.596741 -0.621765  0.391968   -1.230964   
               3.0         0.321960 -0.596741 -0.621765  0.391968   -1.230964   
               4.0         0.321960 -0.596802 -0.621765  0.391907   -1.063105   
...                             ...       ...       ...       ...         ...   
4          3   962.0      -0.749451 -0.095520 -0.654175  0.034607 -167.187300   
               963.0      -0.749451 -0.095520 -0.654175  0.034607 -167.187300   
               964.0      -0.725403 -0.101318 -0.679504  0.041626 -168.474210   
               965.0      -0.725403 -0.101318 -0.679504  0.041626 -168.474210   
               966.0      -0.698059 -0.106567 -0.706360  0.049316 -169.984940   

                                   gy         gz         ax         ay  \
body_label rep sample_num                                                
0          1   0.0           1.063105   0.503576  -0.660156 -10.003906   
               1.0           1.175011   0.000000  -0.660156 -10.003906   
               2.0           1.175011  -0.279765  -0.660156 -10.003906   
               3.0           1.175011  -0.279765  -0.660156 -10.003906   
               4.0           0.839294  -0.727388  -0.660156 -10.003906   
...                               ...        ...        ...        ...   
4          3   962.0      -729.625900 -75.424520 -15.074219  -1.500000   
               963.0      -729.625900 -75.424520 -15.074219  -1.500000   
               964.0      -807.568360 -64.513700 -15.074219  -1.500000   
               965.0      -807.568360 -64.513700 -15.074219  -1.500000   
               966.0      -887.133360 -53.043358 -15.074219  -1.500000   

                                 az  
body_label rep sample_num            
0          1   0.0          1.21875  
               1.0          1.21875  
               2.0          1.21875  
               3.0          1.21875  
               4.0          1.21875  
...                             ...  
4          3   962.0      -11.37500  
               963.0      -11.37500  
               964.0      -11.37500  
               965.0      -11.37500  
               966.0      -11.37500  

[14505 rows x 15 columns]

In [15]:
# Define feature extractors for each sensor

segment_length = 300
interval = 200

extractors = {}
extractors.update(
    {
        channel: [
            FeatureExtractor("mean", np.mean, segment_length=segment_length, interval=interval),
            FeatureExtractor("var", np.var, segment_length=segment_length, interval=interval),
            FeatureExtractor("mean_xing", mean_crossings),
            FeatureExtractor("min", np.min),
            FeatureExtractor("max", np.max),
        ]
        for channel in EMG_CHANNELS
    }
)


extractors.update(
    {
        channel: [
            FeatureExtractor("mean", np.mean, segment_length=segment_length, interval=interval),
            FeatureExtractor("var", np.var, segment_length=segment_length, interval=interval),
            FeatureExtractor("mean_xing", peak_frequency, segment_length=segment_length, interval=interval),
            FeatureExtractor("rmean", lambda x: np.mean(np.abs(x))),
            FeatureExtractor("spec_entropy", spectral_entropy(256, 64))
            
        ]
        for channel in EMG_HP_CHANNELS
    }
)

extractors.update(
    {
        channel: [
            FeatureExtractor("mean", np.mean, segment_length=segment_length, interval=interval),
            FeatureExtractor("var", np.var, segment_length=segment_length, interval=interval),
            FeatureExtractor("min", np.min),
            FeatureExtractor("max", np.max),
            FeatureExtractor("mean_xing", mean_crossings),
            FeatureExtractor("rmean", lambda x: np.mean(np.abs(x))),
            FeatureExtractor('max_pk_pk', lambda x: np.max(x) - np.min(x))

        ]
        for channel in IMU_CHANNELS
    }
)

extractors.update(
    {
        channel: [
            FeatureExtractor("mean", np.mean, segment_length=segment_length, interval=interval),
            FeatureExtractor("var", np.var, segment_length=segment_length, interval=interval),
            FeatureExtractor("mean_xing", mean_crossings),
            FeatureExtractor("rmean", lambda x: np.mean(np.abs(x))),
            FeatureExtractor('max_pk_pk', lambda x: np.max(x) - np.min(x))

      ]
        for channel in GYRO_CHANNELS
    }
)

extractors.update(
    {
        channel: [
            FeatureExtractor("mean", np.mean, segment_length=segment_length, interval=interval),
            FeatureExtractor("var", np.var, segment_length=segment_length, interval=interval),
            FeatureExtractor("min", np.min),
            FeatureExtractor("max", np.max),
            FeatureExtractor("mean_xing", mean_crossings),
            FeatureExtractor("rmean", lambda x: np.mean(np.abs(x))),
            FeatureExtractor('max_pk_pk', lambda x: np.max(x) - np.min(x))
        ]
        for channel in ACC_CHANNELS
    }
)

# Mixed channel features
extractors.update(
    {
        q_pair: [
            FeatureExtractor("corr", lambda x: np.corrcoef(x.T)[0,1])
        ] for q_pair in itertools.combinations(IMU_CHANNELS, 2)
    }
)
extractors.update(
    {
        q_pair: [
            FeatureExtractor("corr", lambda x: np.corrcoef(x.T)[0,1])
        ] for q_pair in itertools.combinations(GYRO_CHANNELS, 2)
    }
)
extractors.update(
    {
        q_pair: [
            FeatureExtractor("corr", lambda x: np.corrcoef(x.T)[0,1])
        ] for q_pair in itertools.combinations(ACC_CHANNELS, 2)
    }
)

In [18]:
results = process_dataframe(dataset, extractors)
results

c.h.0.mean.0.300  c.h.0.mean.200.500  c.h.0.mean.400.700  \
body_label rep                                                             
0.0        1.0          0.802236            0.638268            0.236405   
           2.0         -1.111186            0.673190            0.568211   
           3.0          0.909974           -0.095089           -1.471562   
1.0        1.0          1.058312            0.628681           -0.000771   
           2.0          0.390894            0.619202            0.107014   
           3.0         -1.110074           -0.767111           -2.327882   
2.0        1.0          0.759971            0.455385           -0.002108   
           2.0          0.173244            0.671922            0.150507   
           3.0         -2.056773           -0.814277            1.584465   
3.0        1.0          0.845345            0.144705           -0.336598   
           2.0          0.331531            0.798963           -0.038120   
           3.0         -0.228676           -0.427201            0.704238   
4.0        1.0          0.862126            0.264091           -0.132149   
           2.0         -0.103835            0.283290            1.575691   
           3.0         -1.523089           -3.074019           -0.617342   

                c.h.0.mean.600.900  c.h.0.var.0.300  c.h.0.var.200.500  \
body_label rep                                                           
0.0        1.0            0.269670        -0.583682          -0.690240   
           2.0            0.901030        -0.054467           0.209050   
           3.0           -1.831561        -0.582047           0.127810   
1.0        1.0           -0.226596        -0.581083          -0.688467   
           2.0            0.076032        -0.555173          -0.632793   
           3.0           -2.628022         1.413269           2.044058   
2.0        1.0           -0.004502        -0.569740          -0.676004   
           2.0            0.663593        -0.549489          -0.609401   
           3.0            0.447103         2.563827           2.403388   
3.0        1.0            1.064602        -0.582861          -0.686727   
           2.0            0.213982        -0.556338          -0.587360   
           3.0           -0.303256         0.198374           0.436548   
4.0        1.0            0.050957        -0.587529          -0.620299   
           2.0            0.532881        -0.464676          -0.469728   
           3.0            0.774088         1.491614           0.440165   

                c.h.0.var.400.700  c.h.0.var.600.900  c.h.0.mean_xing  \
body_label rep                                                          
0.0        1.0          -0.756767          -0.715512         0.241633   
           2.0          -0.158889          -0.163269        -0.600185   
           3.0           1.725635           1.729483        -0.857407   
1.0        1.0          -0.755152          -0.708372         1.784966   
           2.0          -0.693095          -0.630707        -0.436498   
           3.0           2.555682           2.326417        -0.950943   
2.0        1.0          -0.753832          -0.717485         0.943148   
           2.0          -0.444460          -0.628266        -0.950943   
           3.0           0.677758           1.240101        -0.693720   
3.0        1.0          -0.754363          -0.583000         0.966532   
           2.0          -0.536209          -0.623775        -0.202660   
           3.0           0.326296           0.516426        -0.296195   
4.0        1.0          -0.695253          -0.720758         2.135724   
           2.0          -0.129454           0.108003        -0.342963   
           3.0           0.392103          -0.429287        -0.740488   

                c.h.0.min  ...  qx.qw.corr  qy.qz.corr  qy.qw.corr  \
body_label rep             ...                                       
0.0        1.0   1.058481  ...    0.564739    0.615669   -1.559685   
           2

In [19]:
from sklearn.decomposition import PCA, KernelPCA
from sklearn.cluster import KMeans

for kernel in ['linear', 'poly', 'rbf', 'sigmoid', 'cosine']:
    for n_components in []:
        print(f'{kernel}.{n_components}')
        pca = KernelPCA(kernel=kernel, n_components=n_components)
        pca.fit(results)
        plt.figure()
        plt.plot(pca.eigenvalues_)
        plt.show()